# **Assignment 4: some database operations**
- **you will learn:** using joins, aggregation, groupby in Pandas
- **task:**  See section 4.2 below
- **deadline:** 17.11.2025
- [Pandas documentation](https://pandas.pydata.org/docs/index.html)
- 📝 **Reminder:** Sync your GitHub repository with the main course repository, update your project in PyCharm, and after completing the assignment, commit and push your changes back to GitHub.

In [1]:
import numpy as np
import pandas as pd

print("NumPy version:", np.__version__)
print("Pandas version:", pd.__version__)

NumPy version: 2.3.3
Pandas version: 2.3.3


In [2]:
# -------------------------------
# 1) Build synthetic datasets
# -------------------------------
# sensor readouts (long-format)
np.random.seed(42)
N_SENSORS = 6
sensors = [f'S{i+1}' for i in range(N_SENSORS)]

start = pd.Timestamp('2025-11-01 00:00')
periods = 24 * 6 # six measurements per hour for 24 hours
freq = '10min' # 10-minute frequency
idx = pd.date_range(start=start, periods=periods, freq=freq)

rows = []
for sensor in sensors:
    base_temp = 15 + 5 * np.random.rand() # baseline temp different per sensor
    base_hum = 40 + 20 * np.random.rand()
    noise_t = np.random.normal(0, 1.5, size=len(idx))
    noise_h = np.random.normal(0, 5.0, size=len(idx))
    # add a diurnal temperature variation
    hours = (idx.hour + idx.minute/60.0)
    diurnal = 6 * np.sin(2 * np.pi * (hours - 6) / 24)
    
    temps = base_temp + diurnal + noise_t
    hums = np.clip(base_hum - 0.5 * diurnal + noise_h, 0, 100)
    
    for ts, t, h in zip(idx, temps, hums):
        rows.append({'sensor_id': sensor, 'timestamp': ts, 'temperature': t, 'humidity': h})

sensor_df = pd.DataFrame(rows)

# add some missing values intentionally and some outliers
sensor_df.loc[sensor_df.sample(frac=0.01, random_state=1).index, 'temperature'] = np.nan
sensor_df.loc[sensor_df.sample(frac=0.005, random_state=2).index, 'humidity'] = np.nan
# inject outliers
outlier_idx = sensor_df.sample(frac=0.002, random_state=3).index
sensor_df.loc[outlier_idx, 'temperature'] += np.random.choice([20, -15], size=len(outlier_idx))

print('--- sensor_df (head) ---')
print(sensor_df.head())

# sensor metadata (wide)
meta = pd.DataFrame({
'sensor_id': sensors,
'location': ['Room A', 'Room B', 'Room C', 'Room A', 'Room D', 'Room B'],
'type': ['thermo-hygro', 'thermo-hygro', 'thermo', 'thermo-hygro', 'hygro', 'thermo']
})

# events table (sparse)
events = pd.DataFrame([
{'sensor_id': 'S1', 'timestamp': start + pd.Timedelta(hours=5), 'event': 'maintenance'},
{'sensor_id': 'S3', 'timestamp': start + pd.Timedelta(hours=8, minutes=20), 'event': 'calibration'},
{'sensor_id': 'S2', 'timestamp': start + pd.Timedelta(hours=18), 'event': 'spike_detected'}
])

--- sensor_df (head) ---
  sensor_id           timestamp  temperature   humidity
0        S1 2025-11-01 00:00:00    11.844233  55.829533
1        S1 2025-11-01 00:10:00    13.162956  55.409148
2        S1 2025-11-01 00:20:00    10.544302  64.612578
3        S1 2025-11-01 00:30:00          NaN  63.473544
4        S1 2025-11-01 00:40:00    13.332673  63.221174


In [3]:
# -------------------------------
# 2) MultiIndex: set index as (sensor_id, timestamp)
# -------------------------------
sensor_mi = sensor_df.set_index(['sensor_id', 'timestamp']).sort_index()
print('\n--- sensor_mi (index sample) ---')
print(sensor_mi.head())

# show selecting using MultiIndex
print('\nSelect all readings for S2 between two times:')
print(sensor_mi.loc['S2'].between_time('06:00', '09:00').head())


--- sensor_mi (index sample) ---
                               temperature   humidity
sensor_id timestamp                                  
S1        2025-11-01 00:00:00    11.844233  55.829533
          2025-11-01 00:10:00    13.162956  55.409148
          2025-11-01 00:20:00    10.544302  64.612578
          2025-11-01 00:30:00          NaN  63.473544
          2025-11-01 00:40:00    13.332673  63.221174

Select all readings for S2 between two times:
                     temperature   humidity
timestamp                                  
2025-11-01 06:00:00    16.292123  38.803557
2025-11-01 06:10:00    16.560178  53.083026
2025-11-01 06:20:00    15.860508  54.342884
2025-11-01 06:30:00    14.793219  44.742754
2025-11-01 06:40:00    16.511539  48.741353


In [4]:
# -------------------------------
# 3) Multi-level columns example: create aggregated summary with MultiIndex columns
# -------------------------------
agg_funcs = {
'temperature': ['mean', 'std', 'min', 'max'],
'humidity': ['mean', 'std']
}
summary_by_sensor = sensor_df.groupby('sensor_id').agg(agg_funcs)
# flatten/unflatten demonstration: keep as MultiIndex columns
print('\n--- summary_by_sensor (MultiIndex columns) ---')
print(summary_by_sensor.head())

# rename columns to readable MultiIndex (level names)
summary_by_sensor.columns.names = ['measurement', 'stat']
print('\nColumns levels:', summary_by_sensor.columns.names)

# access a specific subtable using xs
print('\nTemperature means:')
print(summary_by_sensor.xs('temperature', axis=1)[['mean', 'std']])


--- summary_by_sensor (MultiIndex columns) ---
          temperature                                  humidity          
                 mean       std       min        max       mean       std
sensor_id                                                                
S1          16.718369  4.511676  8.552629  25.213955  59.259889  5.560005
S2          15.427998  4.497470  2.122613  24.023564  45.599224  5.493384
S3          15.658394  4.421943  7.273332  25.556400  56.385015  5.146519
S4          15.668049  4.238744  7.365955  24.204889  48.812680  5.493305
S5          17.801163  4.889507  9.133021  38.594542  49.913718  5.367881

Columns levels: ['measurement', 'stat']

Temperature means:
stat            mean       std
sensor_id                     
S1         16.718369  4.511676
S2         15.427998  4.497470
S3         15.658394  4.421943
S4         15.668049  4.238744
S5         17.801163  4.889507
S6         18.627710  4.633420


In [5]:
summary_by_sensor.columns = [
    f"{measurement}_{stat}"
    for measurement, stat in summary_by_sensor.columns
]

print('\n--- summary_by_sensor (flat columns) ---')
print(summary_by_sensor.head())


--- summary_by_sensor (flat columns) ---
           temperature_mean  temperature_std  temperature_min  \
sensor_id                                                       
S1                16.718369         4.511676         8.552629   
S2                15.427998         4.497470         2.122613   
S3                15.658394         4.421943         7.273332   
S4                15.668049         4.238744         7.365955   
S5                17.801163         4.889507         9.133021   

           temperature_max  humidity_mean  humidity_std  
sensor_id                                                
S1               25.213955      59.259889      5.560005  
S2               24.023564      45.599224      5.493384  
S3               25.556400      56.385015      5.146519  
S4               24.204889      48.812680      5.493305  
S5               38.594542      49.913718      5.367881  


In [6]:
# -------------------------------
# 4) Advanced groupby with named aggregation and custom functions
# -------------------------------
# We'll compute per-sensor and per-location aggregations using named aggregations
sensor_with_meta = sensor_df.merge(meta, on='sensor_id', how='left')

#LEFT = všechny hodnoty z levé tabulky ponechány (SQL joins)

agg_named = sensor_with_meta.groupby(['location']).agg(
temp_mean=('temperature', 'mean'),
temp_med=('temperature', 'median'),
temp_iqr=('temperature', lambda x: np.subtract(*np.percentile(x.dropna(), [75, 25]))),
humidity_mean=('humidity', 'mean'),
n_readings=('temperature', 'count')
)
print('\n--- agg_named by location ---')
print(agg_named)

# custom aggregation using apply (slower but expressive) -> compute time-of-day sensitivity

def day_night_diff(group):
    group = group.copy()
    # daytime mean vs nighttime mean
    group['hour'] = group['timestamp'].dt.hour
    daytime = group.loc[(group['hour'] >= 6) & (group['hour'] < 18), 'temperature']
    nighttime = group.loc[(group['hour'] < 6) | (group['hour'] >= 18), 'temperature']
    return pd.Series({'day_minus_night': daytime.mean() - nighttime.mean()})

sens_daynight = sensor_df.merge(meta, on='sensor_id').groupby('sensor_id').apply(day_night_diff, include_groups=False)
print('\n--- day_minus_night per sensor ---')
print(sens_daynight.head())


--- agg_named by location ---
          temp_mean   temp_med  temp_iqr  humidity_mean  n_readings
location                                                           
Room A    16.187642  16.260233  7.800304      54.036285         283
Room B    17.022280  16.570175  7.810141      48.698361         287
Room C    15.658394  15.917286  8.167460      56.385015         143
Room D    17.801163  18.022922  8.284296      49.913718         142

--- day_minus_night per sensor ---
           day_minus_night
sensor_id                 
S1                7.691988
S2                7.222448
S3                7.461552
S4                7.165945
S5                7.980190


In [7]:
# Note on merge vs join
# pandas.merge() is the core function for all merges/joins.
# df.merge() and df.join() are convenient wrappers:
# - df.join() defaults to left join on df1's index and df2's index.
# - df.merge() defaults to inner join on columns, but can join on indexes.
# Use merge() for flexibility; join() can save typing in simple index-based left joins.

In [8]:
# -------------------------------
# 5) pivot_table & crosstab
# -------------------------------
# pivot_table: average temperature per sensor per hour of day
sensor_df['hour'] = sensor_df['timestamp'].dt.hour
pv = pd.pivot_table(sensor_df, values='temperature', index='sensor_id', columns='hour', aggfunc='mean')
print('\n--- pivot_table mean temperature by sensor x hour ---')
print(pv.iloc[:, :6]) # show first 6 hours as sample

# crosstab: count of readings per sensor per type
sensor_meta_joined = sensor_df.merge(meta, on='sensor_id')
ct = pd.crosstab(sensor_meta_joined['sensor_id'], sensor_meta_joined['type'])
print('\n--- crosstab counts sensor x type ---')
print(ct)


--- pivot_table mean temperature by sensor x hour ---
hour               0          1          2          3          4          5
sensor_id                                                                  
S1         12.210048  10.664322  10.720927  12.989877  14.358660  14.856285
S2          9.690962  10.132154  10.812555  12.572447  13.260633  14.696669
S3         10.254658  10.215027  10.674089  13.038813  12.860999  15.427306
S4          9.326482  10.789089  11.722838  13.088461  13.111283  14.757508
S5         12.241501  12.651350  11.540636  13.750657  14.472117  16.459888
S6         13.411637  12.972633  14.169101  15.001611  16.085027  17.580157

--- crosstab counts sensor x type ---
type       hygro  thermo  thermo-hygro
sensor_id                             
S1             0       0           144
S2             0       0           144
S3             0     144             0
S4             0       0           144
S5           144       0             0
S6             0     144 

In [9]:
# -------------------------------
# 6) Merging and joining examples (all possibilities)
# -------------------------------
# inner merge (only sensors present in both)
m_inner = sensor_df.merge(meta, on='sensor_id', how='inner')
# left merge (keeps all readings)
m_left = sensor_df.merge(meta, on='sensor_id', how='left')
# right and outer
m_right = sensor_df.merge(meta, on='sensor_id', how='right')
m_outer = sensor_df.merge(meta, on='sensor_id', how='outer')

print('\nmerge sizes: inner={}, left={}, right={}, outer={}'.format(
len(m_inner), len(m_left), len(m_right), len(m_outer)
))

# join on index: prepare two DataFrames with indexes
df_a = sensor_df.sample(50, random_state=4).set_index(['sensor_id', 'timestamp']).sort_index()
df_b = pd.DataFrame({'battery': np.random.randint(20, 100, size=30)},
index=df_a.index[:30])
# left join using index
joined = df_a.join(df_b, how='left')

print('\n--- joined (index join) sample ---')
print(joined.head())

# concat: stack vertically and horizontally
concat_v = pd.concat([sensor_df.head(5), sensor_df.tail(5)], axis=0)
concat_h = pd.concat([sensor_df.head(5).reset_index(drop=True).iloc[:, :3],
sensor_df.head(5).reset_index(drop=True).iloc[:, 3:]], axis=1)

# combine_first: combine two sources preferring left's non-null values
left = sensor_df.head(10).copy()
right = left.copy()
right.loc[right.sample(frac=0.3, random_state=5).index, 'temperature'] = np.nan
combined = right.combine_first(left)
print('\n--- combine_first example ---')
print(combined.head())


merge sizes: inner=864, left=864, right=864, outer=864

--- joined (index join) sample ---
                               temperature   humidity  hour  battery
sensor_id timestamp                                                 
S1        2025-11-01 07:50:00    16.998632  51.400346     7     74.0
          2025-11-01 10:30:00    24.450338  75.506305    10     35.0
          2025-11-01 11:00:00    23.210710  60.886517    11     81.0
          2025-11-01 11:10:00    21.762797  59.342354    11     67.0
          2025-11-01 11:40:00    22.796130  52.161576    11     49.0

--- combine_first example ---
  sensor_id           timestamp  temperature   humidity  hour
0        S1 2025-11-01 00:00:00    11.844233  55.829533     0
1        S1 2025-11-01 00:10:00    13.162956  55.409148     0
2        S1 2025-11-01 00:20:00    10.544302  64.612578     0
3        S1 2025-11-01 00:30:00          NaN  63.473544     0
4        S1 2025-11-01 00:40:00    13.332673  63.221174     0


In [10]:
# -------------------------------
# 7) Reshaping: stack/unstack and melt
# -------------------------------
# create a small pivot and show stack/unstack
small = sensor_df[sensor_df['sensor_id'].isin(['S1', 'S2'])].head(12)
pv_small = small.pivot(index='timestamp', columns='sensor_id', values='temperature')
print('\n--- pv_small pivot (timestamp x sensor) ---')
print(pv_small.head())
print('\nstacked -> unstacked roundtrip (demonstration)')
stacked = pv_small.stack()
print(stacked.head())
print(stacked.unstack().head())

# melt: convert wide back to long
melted = pv_small.reset_index().melt(id_vars='timestamp', var_name='sensor_id', value_name='temperature')
print('\n--- melted back to long ---')
print(melted.head())


--- pv_small pivot (timestamp x sensor) ---
sensor_id                   S1
timestamp                     
2025-11-01 00:00:00  11.844233
2025-11-01 00:10:00  13.162956
2025-11-01 00:20:00  10.544302
2025-11-01 00:30:00        NaN
2025-11-01 00:40:00  13.332673

stacked -> unstacked roundtrip (demonstration)
timestamp            sensor_id
2025-11-01 00:00:00  S1           11.844233
2025-11-01 00:10:00  S1           13.162956
2025-11-01 00:20:00  S1           10.544302
2025-11-01 00:40:00  S1           13.332673
2025-11-01 00:50:00  S1           12.166077
dtype: float64
sensor_id                   S1
timestamp                     
2025-11-01 00:00:00  11.844233
2025-11-01 00:10:00  13.162956
2025-11-01 00:20:00  10.544302
2025-11-01 00:40:00  13.332673
2025-11-01 00:50:00  12.166077

--- melted back to long ---
            timestamp sensor_id  temperature
0 2025-11-01 00:00:00        S1    11.844233
1 2025-11-01 00:10:00        S1    13.162956
2 2025-11-01 00:20:00        S1    10.54430

In [11]:
# -------------------------------
# 8) Query: expressive filtering
# -------------------------------
# Use DataFrame.query for readable boolean expressions
q = sensor_df.query('temperature > 25 and humidity < 50')
print(f'\nNumber of readings with temperature>25 and humidity<50: {len(q)}')

# boolean indexing vs query for more complex expressions
expr = "(temperature > temperature.mean()) & (humidity < humidity.quantile(0.25))"
# careful: can't use temperature.mean() inside query easily; compute first
temp_mean = sensor_df['temperature'].mean()
hum_q25 = sensor_df['humidity'].quantile(0.25)
q2 = sensor_df.query('@temp_mean < temperature and humidity < @hum_q25')
print(f'\nUsing external variables in query, matches: {len(q2)}')


Number of readings with temperature>25 and humidity<50: 12

Using external variables in query, matches: 139


In [12]:
# -------------------------------
# 9) Aggregations with groupby + transform and filter
# -------------------------------
# find sensors whose median temperature is > global median
global_median = sensor_df['temperature'].median()
sensor_medians = sensor_df.groupby('sensor_id')['temperature'].median()
hot_sensors = sensor_medians[sensor_medians > global_median].index.tolist()
print('\nSensors with median temperature > global median:', hot_sensors)

# use transform to broadcast group statistic back to rows
sensor_df['sensor_temp_median'] = sensor_df.groupby('sensor_id')['temperature'].transform('median')
# filter rows where temperature is above group's median
above_group_median = sensor_df[sensor_df['temperature'] > sensor_df['sensor_temp_median']]
print('\nRows above their sensor median (sample):')
print(above_group_median.head())

# groupby.filter to keep only sensors with at least 200 readings
sensors_with_enough = sensor_df.groupby('sensor_id').filter(lambda g: len(g) >= 200)
print('\nSensors kept (len>=200):', sensors_with_enough['sensor_id'].unique())


Sensors with median temperature > global median: ['S1', 'S5', 'S6']

Rows above their sensor median (sample):
   sensor_id           timestamp  temperature   humidity  hour  \
29        S1 2025-11-01 04:50:00    17.846883  64.052320     4   
32        S1 2025-11-01 05:20:00    17.064629  58.211946     5   
37        S1 2025-11-01 06:10:00    17.429709  61.295790     6   
38        S1 2025-11-01 06:20:00    18.503335  57.635505     6   
39        S1 2025-11-01 06:30:00    17.912910  62.192710     6   

    sensor_temp_median  
29            17.01069  
32            17.01069  
37            17.01069  
38            17.01069  
39            17.01069  

Sensors kept (len>=200): []


In [13]:
# -------------------------------
# 10) MultiIndex columns: produce aggregated table with hierarchical columns and plot
# -------------------------------
agg_multi = sensor_df.groupby('sensor_id').agg(
temp_mean=('temperature', 'mean'),
temp_std=('temperature', 'std'),
hum_mean=('humidity', 'mean'),
hum_std=('humidity', 'std')
)
# create MultiIndex columns manually
agg_multi.columns = pd.MultiIndex.from_tuples([('temperature', 'mean'), ('temperature', 'std'), ('humidity', 'mean'), ('humidity', 'std')])
print('\n--- agg_multi (MultiIndex columns) ---')
print(agg_multi)


--- agg_multi (MultiIndex columns) ---
          temperature             humidity          
                 mean       std       mean       std
sensor_id                                           
S1          16.718369  4.511676  59.259889  5.560005
S2          15.427998  4.497470  45.599224  5.493384
S3          15.658394  4.421943  56.385015  5.146519
S4          15.668049  4.238744  48.812680  5.493305
S5          17.801163  4.889507  49.913718  5.367881
S6          18.627710  4.633420  51.732933  5.117629


In [14]:
# -------------------------------
# 11) Rolling, expanding and time-based resampling (for sensor S2)
# -------------------------------
s2 = sensor_df[sensor_df['sensor_id'] == 'S2'].set_index('timestamp').sort_index()
# rolling 30-minute window (freq is 10min so 3 observations -> use time-based window)
s2['temp_roll_30min'] = s2['temperature'].rolling('30min').mean()
# humidity diff next reading -> shift(-1)
s2['humidity_next_diff'] = s2['humidity'].shift(-1) - s2['humidity']
# boolean: temp increased >5C and humidity dropped >20 compared to previous measurement
s2['temp_prev_diff'] = s2['temperature'] - s2['temperature'].shift(1)
s2['hum_prev_diff'] = s2['humidity'].shift(1) - s2['humidity']
s2['extreme_change'] = (s2['temp_prev_diff'] < -5) | (s2['hum_prev_diff'] > 20)

## Task Specification: Aggregations, Joins, GroupBy, and Rolling Analysis with Pandas

---

### **Task Overview**
This assignment simulates a **customer purchase analytics workflow** using Pandas. You will work with timestamped transaction data and customer metadata, performing:

- Group-based aggregations
- Time-based analysis
- Rolling window calculations within groups
- Joining customer metadata from another dataset

---

### **Datasets**
You are provided with two datasets:

| File | Description |
|------|-------------|
| `data/purchases.csv` | Timestamped purchase data (contains `Customer_id`, `Timestamp`, `Price`, etc.) |
| `data/customer_info.csv` | Customer metadata (contains `Customer_id`, `Born_date`, `City`, etc.) |

---

### **1. Load and Prepare the Data**
- **Task 1.1:** Load the two CSV files into Pandas DataFrames:
  - Use `df` for `purchases.csv`
  - Use `customer_info` for `customer_info.csv`

- **Task 1.2:** Convert the `Timestamp` column in `df` to Pandas `datetime` format.

- **Task 1.3:** Set `Timestamp` as the **index** of `df` and **sort** the index in ascending order.

---

### **2. Join Customer Metadata**
- **Task 2.1:** Merge the purchase DataFrame (`df`) with the customer metadata DataFrame (`customer_info`) on the `Customer_id` column.

- **Task 2.2:** The resulting DataFrame should include the columns `Born_date` and `City` from the `customer_info` DataFrame.

---

### **3. GroupBy and Aggregations**
Using the merged DataFrame:

- **Task 3.1:** Group by **Customer_id** and compute:
  - Total revenue (`sum(Price)`)
  - Number of purchases (`count`)
  - Average purchase price (`mean(Price)`)

- **Task 3.2:** Create a new feature by extracting the **month name** (e.g., `"January"`, `"February"`) from the `Timestamp`.

- **Task 3.3:** Group by **City** and **Month** and compute:
  - Total monthly revenue per city (`sum(Price)`)

---

### **4. Rolling Window Analysis (Per Customer)**
Perform rolling window calculations **within each customer group**:

- Inside each customer group:
  - **Sort** the DataFrame by `Timestamp`.

- Compute the following:
  - A **3-purchase rolling average** of `Price`.
  - A `diff()` column showing the change in price compared to the previous purchase.

> **Note:** The rolling operations must be computed **within each customer group**, not across all customers.

---

### **5. Advanced Filtering**
- Do the same as in 4 only for customers from **Berlin**, make sure only necessary values are joined (that is filter first, join second).

- Do the same as in 4 only for rows where the `Price` of the current purchase is greater than **20%** than the previous purchase (`Price`), make sure only necessary values are joined (that is filter first, join second).

---

### **6. Custom Lambda Function for GroupBy Operations**
- Apply a **custom lambda function** within a `groupby()` operation to filter or modify the data. For example:
  - Use a lambda to **filter** customers who have purchased more than a specific amount within a given timeframe (e.g., greater than $1000 in total purchases during the last quarter).
  - Apply custom transformations to grouped data using the lambda function.

---

### 7. Time-Based Analysis
- Perform additional time-based analysis:
  - Calculate **monthly growth rate** for total revenue per customer (i.e., compare total revenue of the current month with the previous month).
  - Create a **cumulative sum** of purchases over time per customer.


In [91]:
##Task 1
#Loading data
df = pd.read_csv("data/purchases.csv")
customer_info = pd.read_csv("data/customer_info.csv")

#Indexing
df["Timestamp"] = pd.to_datetime(df["Timestamp"])
df.set_index(df["Timestamp"], inplace = True)
df.sort_index(ascending=True, inplace = True)

In [92]:
##Task 2
#Merging the two datasets
df_new = df.merge(customer_info, on="Customer_id", how="left")

In [93]:
##Task 3
#Decriptive characteristics based on Customer_id
agg_funcs = {"Price": ["sum", "count", "mean"]}
summary_by_customer= df_new.groupby("Customer_id").agg(agg_funcs)
print(summary_by_customer)

#New feature
df_new["Month"] = df_new["Timestamp"].dt.month_name()

#Decriptive characteristics based on City and Month
agg_funcs2 = {"Price": ["sum"]}
summary_by_citymonth= df_new.groupby(["City", "Month"]).agg(agg_funcs2)

                Price                  
                  sum count        mean
Customer_id                            
C001         13055.75    56  233.138393
C002         15927.80    64  248.871875
C003         14947.34    56  266.916786
C004         15111.43    59  256.125932
C005         16798.61    64  262.478281
C006         14774.24    63  234.511746
C007         11886.07    48  247.626458
C008         15555.66    65  239.317846
C009         12630.37    52  242.891731
C010         17554.93    61  287.785738
C011         15360.64    58  264.838621
C012         15547.49    61  254.876885
C013         14548.85    57  255.242982
C014         12758.90    57  223.840351
C015         15948.56    57  279.799298
C016         14485.74    54  268.254444
C017         16645.31    64  260.082969
C018         14808.21    58  255.313966
C019         13761.40    54  254.840741
C020         14716.58    59  249.433559
C021         17883.33    70  255.476143
C022         14539.21    64  227.175156


In [94]:
##Task 4
df_new.sort_values(by = ["Customer_id", "Timestamp"], ascending = [True, True], inplace = True)
df_new["roll_avg_price"] = df_new.groupby("Customer_id")["Price"].rolling(3, min_periods = 1).mean().reset_index(level=0, drop=True)
df_new["Price_diff"] = df_new.groupby("Customer_id")["Price"].diff()

In [95]:
##Task 5
#Berlin customers
berlin_customers = customer_info[customer_info["City"].isin(["Berlin"])]
df_berlin = df[df["Customer_id"].isin(berlin_customers["Customer_id"])]
df_berlin_new = df_berlin.merge(customer_info, on="Customer_id", how="left")
df_berlin_new.sort_values(by = ["Customer_id", "Timestamp"], ascending = [True, True], inplace = True)
df_berlin_new["roll_avg_price"] = df_berlin_new.groupby("Customer_id")["Price"].rolling(3, min_periods = 1).mean().reset_index(level=0, drop=True)
df_berlin_new["Price_diff"] = df_berlin_new.groupby("Customer_id")["Price"].diff()

#Price is higher than 20% of the previous
df_price = df[df["Price"] > df["Price"].shift(1)*1.2]
df_price_new = df_price.merge(customer_info, on="Customer_id", how="left")
df_price_new.sort_values(by = ["Customer_id", "Timestamp"], ascending = [True, True], inplace = True)
df_price_new["roll_avg_price"] = df_price_new.groupby("Customer_id")["Price"].rolling(3, min_periods = 1).mean().reset_index(level=0, drop=True)
df_price_new["Price_diff"] = df_price_new.groupby("Customer_id")["Price"].diff()

In [96]:
##Task 6
df_high_revenue = df_new.groupby("Customer_id").filter(lambda g: g["Price"].sum() >= 1000)
df_high_activity = df_new.groupby("Customer_id").filter(lambda g: len(g) >= 10)

In [97]:
##Task 7
#Montly growth rate
df_monthly = df_new.groupby(["Customer_id", "Month"])["Price"].sum()
df_monthly_rate = df_monthly.groupby("Customer_id").pct_change()

#Cummulative sum
df_new["Price_cumsum"] = df_new.groupby("Customer_id")["Price"].cumsum()